# Time Series Analysis: Alzheimer's disease (No cog)

In [1]:
import pandas as pd 
import numpy as np
import torch

In [ ]:

df=pd.read_csv("/data/df_mock_timeseries_padded.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209100 entries, 0 to 209099
Data columns (total 32 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   NACCID        209100 non-null  object 
 1   visit_num     209100 non-null  int64  
 2   NACCAGE       209100 non-null  int64  
 3   SEX           209100 non-null  int64  
 4   EDUC          209100 non-null  int64  
 5   HYPERTEN      209100 non-null  int64  
 6   DIABETES      209100 non-null  int64  
 7   HYPERCHO      209100 non-null  int64  
 8   CVHATT        209100 non-null  int64  
 9   STROKE        209100 non-null  int64  
 10  TOBAC30       209100 non-null  int64  
 11  ALCOHOL       209100 non-null  int64  
 12  NACCBMI       209100 non-null  float64
 13  NACCFAM       209100 non-null  int64  
 14  NACCALZD      209100 non-null  int64  
 15  VISITYR       209100 non-null  int64  
 16  VISITMO       209100 non-null  int64  
 17  VISITDAY      209100 non-null  int64  
 18  BPSY

In [3]:
import pandas as pd

# 1. Check missing values for each column
print("Missing values per column:")
print(df.isna().sum())

print("\n" + "="*50 + "\n")

# 2. Print numerical statistics for each column
for col in df.columns:
    print(f"Value counts for column: {col}")
    print(df[col].value_counts(dropna=False))
    print("\n" + "-"*40 + "\n")


Missing values per column:
NACCID          0
visit_num       0
NACCAGE         0
SEX             0
EDUC            0
HYPERTEN        0
DIABETES        0
HYPERCHO        0
CVHATT          0
STROKE          0
TOBAC30         0
ALCOHOL         0
NACCBMI         0
NACCFAM         0
NACCALZD        0
VISITYR         0
VISITMO         0
VISITDAY        0
BPSYS           0
COGMODE         0
DEPD            0
MEMORY          0
ORIENT          0
BPDIAS          0
HYPERTEN_BIN    0
DIABETES_BIN    0
HYPERCHO_BIN    0
CVHATT_BIN      0
ALCOHOL_BIN     0
VISIT_DATE      0
delta_days      0
mask            0
dtype: int64


Value counts for column: NACCID
NACCID
NACC999872    10
NACC000034    10
NACC000067    10
NACC000162    10
NACC000176    10
              ..
NACC000919    10
NACC000903    10
NACC000898    10
NACC000818    10
NACC000792    10
Name: count, Length: 20910, dtype: int64

----------------------------------------

Value counts for column: visit_num
visit_num
1     20910
2     20910
3  

## Data Preparation: Long->Tensor

In [4]:
#Choose features
feature_cols = ["NACCAGE", "SEX", "EDUC",  "STROKE",
    "TOBAC30","NACCBMI", "NACCFAM", "BPSYS", "BPDIAS","VISITYR", "VISITMO",
    "VISITDAY","HYPERTEN_BIN", "DIABETES_BIN", "HYPERCHO_BIN", "CVHATT_BIN", "ALCOHOL_BIN",
    "delta_days",]
num_features = len(feature_cols)

# set default sequence length       (record number of visits per subject)
max_seq_len=10          

#Sort the data by id and visit number to ensure chronological order and gather one person's data together
df=df.sort_values(by=["NACCID","visit_num"])

# create a list of unique subjects
subjects = df["NACCID"].unique()
num_subjects = len(subjects)

# Create empty numpy arrays     (Set size in advance to avoid dynamic resizing)     each subject has a 2d array of shape (max_seq_len, num_features)
X_array = np.zeros((num_subjects, max_seq_len, num_features), dtype=np.float32)
masks_array = np.zeros((num_subjects, max_seq_len), dtype=np.bool_)

# Dictionary comprehension to map each NACCID to its index
id_to_index = {id: idx for idx, id in enumerate(subjects)}          

for id, group in df.groupby('NACCID'):
    idx = id_to_index[id]  # Provide the index for the current subject
    
    # Fill in X_array (max_seq_len, num_features)
    X = group[feature_cols].to_numpy()
    
    # Fill in masks_array (num_subjects, max_seq_len)
    m = group['mask'].to_numpy()
    
    length = len(X)
    
    # Copy the data into the preallocated arrays
    X_array[idx, :length, :] = X[:length]
    masks_array[idx, :length] = m[:length]

# 10. Convert numpy arrays to PyTorch tensors
X_tensor = torch.tensor(X_array)           # shape: (num_subjects, max_seq_len, num_features)
mask_tensor = torch.tensor(masks_array)     # shape: (num_subjects, max_seq_len)

# Print the shapes of the tensors 
print("X_tensor shape:", X_tensor.shape)
print("mask_tensor shape:", mask_tensor.shape)

X_tensor shape: torch.Size([20910, 10, 18])
mask_tensor shape: torch.Size([20910, 10])


In [5]:
subjects = df["NACCID"].unique()

grouped = df.groupby('NACCID')

last_indices=mask_tensor.sum(axis=1) - 1

y_list = []

for i, subject_id in enumerate(subjects):
    patient_data = grouped.get_group(subject_id)
    last_visit_index = last_indices[i].item()  # Get the last index for the current subject

    last_alzh=patient_data.iloc[last_visit_index]["NACCALZD"]
    y_list.append(last_alzh)

y_tensor = torch.tensor(y_list, dtype=torch.long)  # shape: (num_subjects,)
print("y_tensor shape:", y_tensor.shape)
print("y_tensor sample:", y_tensor[:10])  # Print first 10 elements for verification")    

class_count=torch.bincount(y_tensor)
print("Class count:", class_count)  # Print the count of each class in y_tensor

y_tensor shape: torch.Size([20910])
y_tensor sample: tensor([1, 0, 1, 0, 1, 0, 1, 0, 1, 0])
Class count: tensor([11580,  9330])


## Define Dataset

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix,roc_auc_score,precision_score,recall_score,f1_score
import numpy as np

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class AlzheimerDataset(Dataset):
    def __init__(self, X_tensor,mask_tensor,y_tensor):          #initialize dataset
        self.X = X_tensor
        self.mask = mask_tensor     
        self.y=y_tensor

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X_item = self.X[idx]
        mask_item = self.mask[idx]
        y_item = self.y[idx]

        # If it is numpy array, change innto tensor
        if isinstance(X_item, np.ndarray):
            X_item = torch.tensor(X_item, dtype=torch.float32)
        if isinstance(mask_item, np.ndarray):
            mask_item = torch.tensor(mask_item, dtype=torch.float32)
        if isinstance(y_item, np.ndarray):
            y_item = torch.tensor(y_item, dtype=torch.long)

        return {
            'X': X_item,
            'mask': mask_item,
            'y': y_item
        }


In [7]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):   # input_dim = number of features    num_layers = number of layers for LSTM model  
        super().__init__()                                               
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers,           # hidden_dim = size of model's hidden state: greater-> more capability, but more chance of overfitting
                            batch_first=True, bidirectional=True)        # tensor shape:(seq_len,batch,input_dim) ->(batch,seq_len,input_dim)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)                  # Apply both past to cuerrent and current to past; Thus, hidden_dim*2

    def forward(self, X, mask):
        out, _ = self.lstm(X)     # (batch, seq_len, hidden_dim*2)  
        lengths = mask.sum(dim=1).long() - 1  # last index for real visit  
        last_h = out[torch.arange(out.size(0)), lengths]  #  last hidden state    (batchNum,last real visit)
        return self.fc(last_h)          #return  group of last sequence for each person

In [8]:
def evaluate(model, data_loader):                                                                  
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():    # Should not change weight or calculate gradients here
        for batch in data_loader:
            X = batch['X'].to(device)
            mask = batch['mask'].to(device)
            y = batch['y'].to(device)

            logits = model(X, mask).squeeze()   # raw output
            probs = torch.sigmoid(logits)       # Turn logits into probs
            preds = (probs >= 0.5).long()       # Binary prediction

            all_preds.append(preds.cpu())       # collect data
            all_labels.append(y.cpu())          
            all_probs.append(probs.cpu())       

    y_true = torch.cat(all_labels).numpy()      # change torch.tensor to numpy arrays
    y_pred = torch.cat(all_preds).numpy()
    y_prob = torch.cat(all_probs).numpy()

    acc = accuracy_score(y_true, y_pred)                         
    auc = roc_auc_score(y_true, y_prob)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    return acc, auc, precision, recall, f1,y_true, y_pred

# Accuracy: % of correct predictions

# AUC: How well the model ranks positive samples higher than negative ones

# Precision: Among predicted positives, how many were correct?

# Recall: Among actual positives, how many did we catch?

# F1: Balance between precision and recall

In [9]:
# --- Data Preparation (X_tensor, mask_tensor, y_tensor) ---

dataset = AlzheimerDataset(X_tensor, mask_tensor, y_tensor)                 # Load the data
batch_size = 32
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

num_neg = (y_tensor == 0).sum().item()                                # Handle the imbalance
num_pos = (y_tensor == 1).sum().item()
pos_weight_val = num_neg / num_pos
pos_weight = torch.tensor([pos_weight_val], dtype=torch.float32).to(device)
print("pos_weight:", pos_weight.item())

input_dim = X_tensor.size(2)                                       # num of features
hidden_dim = 64
num_layers = 1
output_dim = 1

model = LSTMClassifier(input_dim, hidden_dim, num_layers, output_dim).to(device)      # modl initialization
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)    # weight on pos to handle imbalance
optimizer = optim.Adam(model.parameters(), lr=1e-4)    # adjust weight during training


pos_weight: 1.2411575317382812


In [ ]:
num_epochs = 20
best_recall = 0
best_metrics = {}

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        x_batch = batch['X'].to(device)             # (batch, seq_len, features)
        m_batch = batch['mask'].to(device)          # (batch, seq_len)
        y_batch = batch['y'].float().to(device).view(-1,1)  # (batch, 1)

        optimizer.zero_grad()
        logits = model(x_batch, m_batch)             # (batch, output_dim)

        # Calculate loss
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * x_batch.size(0)

    avg_loss = total_loss / len(dataset)
    print(f"Epoch {epoch+1}/{num_epochs} — Loss: {avg_loss:.4f}")

    acc, auc, prec, rec, f1, y_true, y_pred = evaluate(model, train_loader)
    print(f"Train Accuracy: {acc:.4f}, AUC: {auc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}")

    if rec > best_recall and prec > 0.5:
        best_recall = rec
        best_metrics = {
            'acc': acc,
            'auc': auc,
            'precision': prec,
            'recall': rec,
            'f1': f1,
            'y_true': y_true,
            'y_pred': y_pred
        }
        torch.save(model.state_dict(), 'best_model_by_recall2.pt')

print("\nConfusion Matrix (Best Model):")
print(confusion_matrix(best_metrics['y_true'], best_metrics['y_pred']))

print("\nClassification Report (Best Model):")
print(classification_report(best_metrics['y_true'], best_metrics['y_pred'], digits=4))

print("\nBest Train Performance (by Recall):")
print(f"Accuracy: {best_metrics['acc']:.4f}")
print(f"AUC: {best_metrics['auc']:.4f}")
print(f"Precision: {best_metrics['precision']:.4f}")
print(f"Recall: {best_metrics['recall']:.4f}")
print(f"F1 Score: {best_metrics['f1']:.4f}")

model.load_state_dict(torch.load('best_model_by_recall2.pt'))
model.eval()


Epoch 1/20 — Loss: 0.7679
Train Accuracy: 0.5602, AUC: 0.5738, Precision: 0.5070, Recall: 0.5218, F1: 0.5143
Epoch 2/20 — Loss: 0.7612
Train Accuracy: 0.5623, AUC: 0.5998, Precision: 0.5073, Recall: 0.6662, F1: 0.5760
Epoch 3/20 — Loss: 0.7561
Train Accuracy: 0.5738, AUC: 0.5984, Precision: 0.5198, Recall: 0.5871, F1: 0.5514
Epoch 4/20 — Loss: 0.7530
Train Accuracy: 0.5806, AUC: 0.6167, Precision: 0.5254, Recall: 0.6214, F1: 0.5694
Epoch 5/20 — Loss: 0.7477
Train Accuracy: 0.5872, AUC: 0.6210, Precision: 0.5347, Recall: 0.5768, F1: 0.5550
Epoch 6/20 — Loss: 0.7448
Train Accuracy: 0.5946, AUC: 0.6336, Precision: 0.5403, Recall: 0.6133, F1: 0.5745
Epoch 7/20 — Loss: 0.7420
Train Accuracy: 0.6073, AUC: 0.6404, Precision: 0.5703, Recall: 0.4859, F1: 0.5247
Epoch 8/20 — Loss: 0.7400
Train Accuracy: 0.6032, AUC: 0.6431, Precision: 0.5499, Recall: 0.6102, F1: 0.5785
Epoch 9/20 — Loss: 0.7373
Train Accuracy: 0.6095, AUC: 0.6453, Precision: 0.5786, Recall: 0.4594, F1: 0.5121
Epoch 10/20 — Loss:

LSTMClassifier(
  (lstm): LSTM(18, 64, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
)